## 아동 보육시설 전처리

### 데이터 전처리 (Data Preprocessing)

이 노트북은 서울시 아동 보육시설 데이터를 분석에 적합한 형태로 전처리하기 위해 작성되었습니다.

- 데이터 로드 및 기본 정보 확인
- 운영현황을 기준으로 데이터 필터링
- 구별 데이터 통합 및 정제

전처리를 통해 모델링에 필요한 깨끗하고 일관된 형태의 데이터셋을 구축하는 데 목적이 있습니다.

### 1. 데이터 로드 및 기본 설정

In [1]:
#  Jupyter Notebook에서 외부 모듈(.py 파일)을 수정할 경우,
#  매번 커널을 재시작하거나 수동으로 reload하지 않아도
#  자동으로 가장 최신 상태로 import 되도록 설정합니다.

# %load_ext autoreload:
# IPython의 autoreload 확장 기능을 불러옵니다.
# 이 기능은 외부 .py 파일이 변경될 때 자동으로 다시 import 하도록 도와줍니다.

# %autoreload 2:
# 모든 모듈을 cell 실행 시마다 자동으로 reload합니다.
# (즉, 외부 .py 파일을 수정하고 저장만 해도 바로 반영됨)

# 사용 예:
#   - utils.py, visualization.py 등 자주 수정하는 모듈이 있는 경우
#   - 매번 커널 재시작 없이 편리하게 함수 변경 사항을 반영하고 싶을 때

# 단, 모듈의 내부 상태를 유지하고 싶은 경우에는 주의해서 사용해야 합니다.

%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 프로젝트 루트 경로를 sys.path에 추가
project_root = os.path.abspath("..")  # notebooks 폴더 기준 상위 폴더
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from scripts.utils import setup_font

# 한글 폰트 설정
setup_font()

Current OS: Darwin
Setting macOS font: AppleGothic
Current font settings: ['AppleGothic']


In [4]:
import glob
# 원본 데이터가 저장된 폴더 경로 설정
folder_path = '../data/raw/daycarecenter'
save_dir = '../data/processed/daycarecenter'

# 처리할 서울시 어린이집 데이터 CSV 파일 경로 가져오기
files = glob.glob(os.path.join(folder_path, '*.csv'))

# 저장 폴더 생성
os.makedirs(save_dir, exist_ok=True)

# 결과를 저장할 빈 DataFrame 생성
daycarecenter_merged_df = pd.DataFrame()

### 2. 구별 어린이집 데이터 기본 전처리 및 통합
- 열 이름 정제
- 운영현황을 기준으로 데이터 필터링
- 필요한 컬럼 선택
- 통합 데이터 저장

In [5]:
from scripts.utils import save_to_csv, strip_column_names

# 각 구별 데이터 처리
for file in files:
    # CSV 파일 읽기
    df = pd.read_csv(file, encoding='utf-8')
    
    # 열 이름 정제
    df = strip_column_names(df)
        
    # "운영현황"이 "정상" 또는 "재개"인 데이터만 필터링
    df = df[df['운영현황'].isin(['정상', '재개'])]
    
    # 필요한 컬럼만 선택
    df = df[['시군구', '어린이집명', '주소', '위도', '경도']]
    
    # 결과 DataFrame에 추가
    daycarecenter_merged_df = pd.concat([daycarecenter_merged_df, df], ignore_index=True)

### 3. 통합된 어린이집 데이터와 측정소 위치 데이터 매핑
- 어린이집과 측정소의 위도 경도 숫자형 변환
- 어린이집에 대해 가장 가까운 측정소 검색
- 측정소, 측정소까지의 거리 컬럼 추가

In [6]:
# 위도와 경도 좌표를 사용하여 두 지점 사이 거리를 계산하기 위해 haversine 라이브러리 설치
# pip install haversine==2.9.0

In [7]:

import numpy as np
from haversine import haversine

# 데이터 읽기
station_df = pd.read_csv('../data/raw/monitoringStation/seoul_monitoring_stations.csv', encoding='utf-8')

# 위도, 경도를 숫자형으로 변환
daycarecenter_merged_df['위도'] = pd.to_numeric(daycarecenter_merged_df['위도'], errors='coerce')
daycarecenter_merged_df['경도'] = pd.to_numeric(daycarecenter_merged_df['경도'], errors='coerce')
station_df['위도'] = pd.to_numeric(station_df['위도'], errors='coerce')
station_df['경도'] = pd.to_numeric(station_df['경도'], errors='coerce')

# 측정소 좌표 리스트 생성
station_coords = list(zip(station_df['위도'], station_df['경도']))
station_names = station_df['측정소명'].tolist()

# 각 어린이집에 대해 가장 가까운 측정소 찾기
nearest_stations = []
nearest_distances = []

for idx, row in daycarecenter_merged_df.iterrows():
    if pd.isna(row['위도']) or pd.isna(row['경도']):
        nearest_stations.append(None)
        nearest_distances.append(None)
        continue
        
    daycare_coord = (row['위도'], row['경도'])
    
    # 모든 측정소와의 거리 계산
    distances = [haversine(daycare_coord, station_coord, unit='km') for station_coord in station_coords]
    
    # 가장 가까운 측정소 찾기
    min_idx = np.argmin(distances)
    nearest_stations.append(station_names[min_idx])
    nearest_distances.append(distances[min_idx])

# 결과를 DataFrame에 추가
daycarecenter_merged_df['측정소'] = nearest_stations
daycarecenter_merged_df['측정소까지거리(km)'] = nearest_distances

# 결과를 CSV 파일로 저장
save_to_csv(daycarecenter_merged_df, output_dir= save_dir, file_name='daycarecenter_preprocessed')

daycarecenter_merged_df.head()

,시군구,어린이집명,주소,위도,경도,측정소,측정소까지거리(km)
0,구로구,명화어린이집,서울특별시 구로구 개봉로17다길 6 (개봉동),37.493450,126.855103,구로구,3.134825
1,구로구,꿈이있는어린이집,서울특별시 구로구 벚꽃로 484 화원사회종합복지관내 (구로동),37.497329,126.880142,구로구,0.885956
2,구로구,이편한어린이집,서울특별시 구로구 신도림로 16 대림아파트 관리동1층 (신도림동),37.505801,126.880712,구로구,1.179900
3,구로구,별나라어린이집,"서울특별시 구로구 신도림로11가길 36 5동 101호(신도림동, 미성아파트)",37.508274,126.876108,구로구,1.663649
4,구로구,미래어린이집,서울특별시 구로구 구로동로20길 57 (구로동),37.488049,126.887631,구로구,1.155982
